In [1]:
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco
import string
import os
import numpy as np

In [2]:
path = '/Users/ariana/desktop/historical_tech'
os.chdir(path)

In [3]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-global-statistics-mineral-and-material


In [4]:
def read_usgs_896(tech_name, file_name, sheet_name=0):
    df = pd.read_excel(file_name, sheet_name=sheet_name, header=3, na_values=['XX','--', 'W', '(3/)'])
    drop_idx = []
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
        if country in drop_list:
            drop_idx.append(idx)
    df.drop(index=drop_idx, inplace=True)
    df.reset_index(drop=True, inplace=True)
    for idx in df.index:
        country = df[df.columns[0]].iloc[idx]
        if country == 'World':
            world_idx = idx + 1
    df = df[:world_idx]
    countries = []
    for country in df[df.columns[0]]:
        country = country.strip(string.digits)
        country = country.strip()
        countries.append(country)
    df['Country Name'] = countries
    df.drop(columns=df.columns[0], inplace=True)
    iso2 = []
    spatial_scale = []
    for country in df['Country Name']:
        if country == 'Serbia and Montenegro':
            iso2.append('CS')
            spatial_scale.append('National')
        elif country == 'U.S.S.R.':
            iso2.append('SU')
            spatial_scale.append('National')
        elif country == 'Yugoslavia':
            iso2.append('YU')
            spatial_scale.append('National')
        elif country == 'World':
            iso2.append('World')
            spatial_scale.append('Global')
        elif country == 'Netherlands Antilles':
            iso2.append('AN')
            spatial_scale.append('National')
        else:
            iso2.append(coco.convert(names=country, to='iso2'))
            spatial_scale.append('National')
    df['Country Code'] = iso2
    df['Spatial Scale'] = spatial_scale
    df['Data Source'] = 'U.S. Geological Survey'
    df['Unit'] = 'Metric tons'
    df['Metric'] = 'Annual production'
    df['Technology Name'] = tech_name
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    df.set_index('ID', inplace=True)
    file_name = 'cleaned data/ds896_' + tech_name + '.csv'
    df.to_csv(file_name)
    return df


In [5]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-global-statistics-mineral-and-material#:~:text=Aluminum-,ds896%2Daluminum.xlsx,-1990%2D2012
# Aluminum and Bauxite


In [6]:
aluminum = read_usgs_896('Primary aluminum production', 'raw data/usgs/ds896-aluminum.xlsx', 'Aluminum, primary production')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [7]:
bauxite = read_usgs_896('Primary bauxite production', 'raw data/usgs/ds896-aluminum.xlsx', 'Bauxite')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [8]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-cadmium.xlsx
# Cadmium


In [9]:
cadmium = read_usgs_896('Cadmium Refining', 'raw data/usgs/ds896-cadmium.xlsx')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [10]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-copper.xlsx
# Copper mining and refining


In [11]:
copper_mining = read_usgs_896('Copper Mining', 'raw data/usgs/ds896-copper.xlsx', 'Mine')
copper_mining = copper_mining[copper_mining['    Mine production']=='Mine: Total']
copper_mining.drop(columns='    Mine production', inplace=True)
copper_mining.to_csv('cleaned data/ds896_Copper Mining.csv')


In [12]:
copper_refining = read_usgs_896('Copper Refining', 'raw data/usgs/ds896-copper.xlsx', 'Refinery')
copper_refining = copper_refining[copper_refining['Refinery production']=='Refinery: Total']
copper_refining.drop(columns='Refinery production', inplace=True)
copper_refining.to_csv('cleaned data/ds896_Copper Refining.csv')


In [13]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-iron-steel.xlsx
# Iron ore and raw steel


In [14]:
iron = read_usgs_896('Iron ore', 'raw data/usgs/ds896-iron-steel.xlsx', 'Iron ore, gross weight')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [15]:
steel = read_usgs_896('Raw steel production', 'raw data/usgs/ds896-iron-steel.xlsx', 'Raw steel')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [16]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-lead.xlsx
# Lead


In [17]:
lead = read_usgs_896('Lead mine production', 'raw data/usgs/ds896-lead.xlsx', 'Mine')


/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [ ]:
# https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/mineral-pubs/historical-statistics/global/ds896-salt.xlsx
# Salt


In [18]:
salt = pd.read_excel('raw data/usgs/ds896-salt.xlsx', sheet_name=0, header=3, na_values=['XX','--', 'W', '(3/)'])
drop_idx = []
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    drop_list = ['Limonite', 'Other', 'Puerto Rico', 'United States: Puerto Rico', 'Leeward and Windward Islands ']
    if country in drop_list:
        drop_idx.append(idx)
salt.drop(index=drop_idx, inplace=True)
salt.reset_index(drop=True, inplace=True)
for idx in salt.index:
    country = salt[salt.columns[0]].iloc[idx]
    if country == 'World':
        world_idx = idx + 1
salt = salt[:world_idx]
countries = []
for country in salt[salt.columns[0]]:
    country = country.strip(string.digits)
    country = country.strip()
    countries.append(country)
salt['Country Name'] = countries
salt.drop(columns=[salt.columns[0], 'Salt type'], inplace=True)
unique_countries = sorted(list(set(salt['Country Name'])))
salt_type_sum = []
for country in unique_countries:
    dup_df = salt[salt['Country Name']==country].sum()
    dup_df = dup_df[:-1]
    val_list = []
    for x in dup_df:
        val_list.append(x)
    val_list.append(country)
    salt_type_sum.append(val_list)
salt = pd.DataFrame(salt_type_sum, columns=salt.columns)
iso2 = []
spatial_scale = []
for country in salt['Country Name']:
    if country == 'Serbia and Montenegro':
        iso2.append('CS')
        spatial_scale.append('National')
    elif country == 'U.S.S.R.':
        iso2.append('SU')
        spatial_scale.append('National')
    elif country == 'Yugoslavia':
        iso2.append('YU')
        spatial_scale.append('National')
    elif country == 'World':
        iso2.append('World')
        spatial_scale.append('Global')
    elif country == 'Netherlands Antilles':
        iso2.append('AN')
        spatial_scale.append('National')
    else:
        iso2.append(coco.convert(names=country, to='iso2'))
        spatial_scale.append('National')
salt['Country Code'] = iso2
salt['Spatial Scale'] = spatial_scale
salt['Data Source'] = 'U.S. Geological Survey'
salt['Unit'] = 'Metric tons'
salt['Metric'] = 'Annual production'
salt['Technology Name'] = 'Salt production'
salt['ID'] = salt['Technology Name'] + '_' + salt['Metric'] + '_' + salt['Country Code']
salt.set_index('ID', inplace=True)
salt.replace(0, np.nan, inplace=True)
salt.to_csv('cleaned data/ds896_Salt production.csv')


In [ ]:
# https://www.usgs.gov/centers/national-minerals-information-center/historical-statistics-mineral-and-material-commodities
# Data Series 140


In [ ]:
# https://www.usgs.gov/media/files/cement-historical-statistics-data-series-140
# Cement

In [ ]:
# cement = pd.read_excel('raw data/usgs/ds140-cement-2019.xlsx', header=4, 
#                        usecols=[0,1,7,8])
# for idx in cement.index:
#     year = cement['Year'].iloc[idx]
#     if year == 2019:
#         last_row = idx + 1
# cement = cement[:last_row]
# cement.set_index('Year', inplace=True)
# cement = cement.transpose()
# cement

In [ ]:
# https://www.usgs.gov/media/files/copper-historical-statistics-data-series-140
# Primary copper

In [ ]:
# https://www.usgs.gov/media/files/gold-historical-statistics-data-series-140
# Gold

In [ ]:
# https://www.usgs.gov/media/files/nickel-historical-statistics-data-series-140
# Nickel

In [ ]:
# https://www.usgs.gov/media/files/construction-sand-and-gravel-historical-statistics-data-series-140
# Sand and gravel (construction)


In [ ]:
# https://www.usgs.gov/media/files/industrial-sand-and-gravel-historical-statistics-data-series-140
# Sand and gravel (industrial)
